### Imports and settings

In [ ]:
import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np

%matplotlib widget
from aeromaps import create_process
from aeromaps.core.models import (
    default_models_bottom_up,
)

from aeromaps.models.air_transport.aircraft_fleet_and_operations.fleet.fleet_model import (
    AircraftParameters,
    Aircraft,
)


plt.rc("hatch", linewidth=1)

# 1 - Scenario initialisation and settings

The first phase corresponds to running a first, central scenario with AeroMAPS. 

In [ ]:
models = {
    "default_models_bottom_up": default_models_bottom_up,
}

In [ ]:
process = create_process(
    models=models,
    use_fleet_model=True,
    add_examples_aircraft_and_subcategory=False,
)

## 1-a) Air traffic evolution

As of April 2024, AeroMAPS is considering an exgenous growth rate.
For the purposes of this article, an uniform growth rate of 3% is set. This is significantly lower from industry prospects such as [Airbus GMF: 3.6%](https://www.airbus.com/en/products-services/commercial-aircraft/market/global-market-forecast) or [Boeing CMO: 3.8%](https://www.boeing.com/content/dam/boeing/boeingdotcom/market/assets/downloads/CMO_2022_Report_FINAL_v02.pdf), but higher than [IEA Net Zero: 2.1% (2019 as base year)](https://www.iea.org/reports/net-zero-roadmap-a-global-pathway-to-keep-the-15-0c-goal-in-reach).


In [ ]:
## Growth rate by category [%]
process.parameters.cagr_passenger_short_range_reference_periods = []
process.parameters.cagr_passenger_short_range_reference_periods_values = [3.0]
process.parameters.cagr_passenger_medium_range_reference_periods = []
process.parameters.cagr_passenger_medium_range_reference_periods_values = [3.0]
process.parameters.cagr_passenger_long_range_reference_periods = []
process.parameters.cagr_passenger_long_range_reference_periods_values = [3.0]
process.parameters.cagr_freight_reference_periods = []
process.parameters.cagr_freight_reference_periods_values = [3.0]

## 1-b) Aircraft fleet

**Hypotheses considered for aircraft DOC and new aircraft introduction:**
The previously defined air traffic is flown by the default AeroMAPS fleet, and new aircraft. 

- A default fleet: an "old" and a "recent" aircraft for three markets: Short (<1500 km), Medium (1500-4000 km) and Long Range (>4000 km). 
- New aircraft are introduced
    - <u>Short Range:</u>
        - Drop-in aircraft in 2035
    - <u>Medium Range:</u>
        - Drop-in aircraft in 2035
    - <u>Long Range:</u>
        - Drop-in aircraft in 2040
        
    - Efficiency assumptions: drop in aircraft introduced in 2030 use 20% less energy than the current ("recent"), the second long-range dropin aircraft introduced in 2045 uses 30% less energy than the "recent. Hydrogen aircraft uses the same ammount of energy than the recent aircraft.
    - Aircraft RC and NRC cost assumption are based on [this article](https://www.eucass.eu/doi/EUCASS2023-593.pdf), but these parameter are not used in this article. There is currently <u>no automated link between RC+NRC and DOC on AeroMAPS </u>. Consistent values are filed nevertheless. Same for ASK per year per A/C.
    
    
    - Aircraft DOC are obtained using the same article and regrouped in two categories
        - DOC-ENERGY is computed using the energy consumption of the aircraft and the MFSP of the corresponding energy (see below). 
        - DOC-NON-ENERGY encompass all other direct operating Costs (Navigation, FA/Pilots, MRO,...). AeroMAPS requires the evolution of this parameter compared to the reference. For  information, the capital part represent around 16% of the total DOC-NE and other costs 84%. A composite evolution can therefore be computed. The capital part is obtained (outside of AeroMAPS) by computing an equivalent annual cost (**EAC**), using an annuity factor of 0.07 (computed for 25 years, under a 5% Interest rate). It can be modified to represent a increase in aircraft cost or lifespan. ($\text{EAC} = \frac{CAP_{t0} \times i}{1 - (1 + i)^{-n}} $). For the time being, the DOC-NE of drop-,n aircraft is left unchanged, and the DOC NE of both hydrogen aircraft is set at 106% and 110% of the basis according to [Hoelzen et al.](https://www.sciencedirect.com/science/article/pii/S0360319921043184#fig2)
        
    
    
- Aircraft fleet renewal rates are set to 25 years. It corresponds to both the aircraft life and the duration necessary to renew the whole fleet with current fleet renewal models of AeroMAPS.






In [ ]:
# Aircraft fleet

## Initialization
fleet = process.fleet

## Short Range aircraft
short_range_aircraft_params_di = AircraftParameters(
    entry_into_service_year=2035,
    consumption_evolution=-20.0,
    nox_evolution=0.0,
    soot_evolution=0.0,
    doc_non_energy_evolution=0.0,
    cruise_altitude=12000.0,
    ask_year=352000000.0,
    rc_cost=60000000.0,
    nrc_cost=10000000000.0,
)
short_range_aircraft_di = Aircraft(
    "New SR-DI", parameters=short_range_aircraft_params_di, energy_type="DROP_IN_FUEL"
)
fleet.categories["Short Range"].subcategories[0].add_aircraft(aircraft=short_range_aircraft_di)

## Medium Range
medium_range_aircraft_params_di = AircraftParameters(
    entry_into_service_year=2035,
    consumption_evolution=-20.0,
    nox_evolution=0.0,
    soot_evolution=0.0,
    doc_non_energy_evolution=0.0,
    cruise_altitude=12000.0,
    ask_year=352000000.0,
    rc_cost=60000000.0,
    nrc_cost=10000000000.0,
)
medium_range_aircraft_di = Aircraft(
    "New MR-DI", parameters=medium_range_aircraft_params_di, energy_type="DROP_IN_FUEL"
)
fleet.categories["Medium Range"].subcategories[0].add_aircraft(aircraft=medium_range_aircraft_di)


## Long Range
long_range_aircraft_params_di1 = AircraftParameters(
    entry_into_service_year=2040,
    consumption_evolution=-30.0,
    nox_evolution=0.0,
    soot_evolution=0.0,
    doc_non_energy_evolution=0.0,
    cruise_altitude=12000.0,
    ask_year=912000000.0,
    rc_cost=150000000.0,
    nrc_cost=25000000000.0,
)
long_range_aircraft_di1 = Aircraft(
    "New LR-DI", parameters=long_range_aircraft_params_di1, energy_type="DROP_IN_FUEL"
)
fleet.categories["Long Range"].subcategories[0].add_aircraft(aircraft=long_range_aircraft_di1)


## Fleet renewal
fleet.categories["Short Range"].parameters.life = 25
fleet.categories["Medium Range"].parameters.life = 25
fleet.categories["Long Range"].parameters.life = 25

In [ ]:
process.compute()
process.write_json()

In [ ]:
process.data["vector_outputs"]["energy_per_ask_mean"].plot()

(
    process.data["vector_outputs"]["energy_per_ask_mean"][2050]
    / process.data["vector_outputs"]["energy_per_ask_mean"][2021]
) ** (1 / (2050 - 2021)) - 1

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Extract the data series
energy_per_ask = pd.Series(process.data["vector_outputs"]["energy_per_ask_mean"])

# Set time range and CAGR calculation
start_year = 2021
end_year = 2050
years = np.arange(start_year, end_year + 1)

# Actual values for those years (in case the series is longer or missing some)
actual = energy_per_ask.loc[years]

# Calculate CAGR
cagr = (actual[end_year] / actual[start_year]) ** (1 / (end_year - start_year)) - 1

# Generate the CAGR curve
cagr_curve = actual[start_year] * (1 + cagr) ** (years - start_year)

# Plot
plt.figure(figsize=(6, 4))
plt.plot(energy_per_ask.index, energy_per_ask.values, linestyle="-", label="E/ASK fleet model")
plt.plot(years, cagr_curve, linestyle="--", color="orange", label=f"CAGR ({cagr*100:.2f}%/yr)")

# Annotate CAGR
plt.text(
    end_year,
    cagr_curve[-1] + 0.15,
    f"CAGR: {cagr*100:.2f}%",
    verticalalignment="bottom",
    horizontalalignment="right",
    fontsize=10,
    color="darkorange",
)

# Formatting
plt.title("Energy per ASK with fleet model vs simple CAGR", fontsize=14)
plt.xlabel("Year")
plt.ylabel("Energy per ASK (MJ/ASK)")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()